In [7]:
from typing import Annotated, Literal, TypedDict
from langgraph.graph import END
from langgraph.graph.state import StateGraph
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain.chat_models import init_chat_model
from datetime import datetime
from dotenv import load_dotenv
from langgraph.graph.message import add_messages
import os

In [2]:
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

In [4]:
llm = init_chat_model("groq:gemma2-9b-it", temperature=0)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x128f02110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x128f031d0>, model_name='gemma2-9b-it', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
class OrgState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    next_agent: Literal["ceo","research_lead","data_researcher","market_researcher","writing_lead","tech_writer","summary_writer","end"] | None

In [9]:
def ceo(state: OrgState) -> dict:
    """CEO coordinates team leaders; deterministic progression prevents loops."""
    msgs = state.get("messages", [])
    task = msgs[-1].content if msgs else state.get("current_task", "")

    # Decide stage by stage
    if not state.get("research_synthesized", False):
        msg = "CEO: Handing off to Research Team Leader for research."  # -> research_lead
        nxt = "research_lead"
    elif not (state.get("tech_draft_done", False) and state.get("summary_done", False)):
        msg = "CEO: Handing off to Writing Team Leader for drafting & summary."  # -> writing_lead
        nxt = "writing_lead"
    else:
        # Assemble final report once both drafts present
        final = (
            "FINAL REPORT\n" + "=" * 60 + "\n" +
            f"Generated: {datetime.now().isoformat()}\n" +
            f"Topic: {task}\n" + "=" * 60 + "\n\n" +
            "Research Synthesis:\n" + state.get("research_synthesis", "") + "\n\n" +
            "Technical Draft:\n" + state.get("tech_draft", "") + "\n\n" +
            "Executive Summary:\n" + state.get("summary_text", "") + "\n"
        )
        return {
            "messages": [AIMessage(content="CEO: Collected all artifacts. Finalizing report.")],
            "final_report": final,
            "task_complete": True,
            "next_agent": "end",
            "current_task": task,
        }

    return {
        "messages": [AIMessage(content=msg)],
        "next_agent": nxt,
        "current_task": task,
    }


In [10]:
def research_lead(state: OrgState) -> dict:
    """Orchestrates Data & Market researchers, then synthesizes findings."""
    task = state.get("current_task", "")
    data_done = state.get("data_done", False)
    market_done = state.get("market_done", False)

    if not data_done:
        return {
            "messages": [AIMessage(content="Research Lead: Sending to Data Researcher.")],
            "next_agent": "data_researcher",
            "current_task": task,
        }
    if not market_done:
        return {
            "messages": [AIMessage(content="Research Lead: Sending to Market Researcher.")],
            "next_agent": "market_researcher",
            "current_task": task,
        }

    # Both complete -> synthesize once
    if not state.get("research_synthesized", False):
        synthesis_prompt = (
            "You are a Research Team Leader. Synthesize the two briefs into a concise,\n"
            "actionable research summary with citations and contradictions called out.\n\n"
            f"Task: {task}\n\n"
            f"DATA NOTES:\n{state.get('data_notes','')}\n\n"
            f"MARKET NOTES:\n{state.get('market_notes','')}\n"
        )
        out = llm.invoke([HumanMessage(content=synthesis_prompt)]).content
        return {
            "messages": [AIMessage(content="Research Lead: Synthesis completed.")],
            "research_synthesis": out,
            "research_synthesized": True,
            "next_agent": "ceo",
        }

    # Already synthesized -> return to CEO
    return {
        "messages": [AIMessage(content="Research Lead: Research package ready. Returning to CEO.")],
        "next_agent": "ceo",
    }

In [11]:
def data_researcher(state: OrgState) -> dict:
    task = state.get("current_task", "")
    prompt = (
        "You are a Data Researcher. Provide data-driven facts: stats, studies,\n"
        "benchmarks, methodologies. Cite inline. Keep it tight (200-300 words).\n\n"
        f"Topic: {task}"
    )
    out = llm.invoke([HumanMessage(content=prompt)]).content
    return {
        "messages": [AIMessage(content="Data Researcher: Provided data brief.")],
        "data_notes": out,
        "data_done": True,
        "next_agent": "research_lead",
    }

In [12]:
def market_researcher(state: OrgState) -> dict:
    task = state.get("current_task", "")
    prompt = (
        "You are a Market Researcher. Provide market landscape: segments, TAM/SAM,\n"
        "competitors, pricing, trends, regulatory factors. Cite inline. 200-300 words.\n\n"
        f"Topic: {task}"
    )
    out = llm.invoke([HumanMessage(content=prompt)]).content
    return {
        "messages": [AIMessage(content="Market Researcher: Provided market brief.")],
        "market_notes": out,
        "market_done": True,
        "next_agent": "research_lead",
    }

In [13]:
def writing_lead(state: OrgState) -> dict:
    task = state.get("current_task", "")
    if not state.get("tech_draft_done", False):
        return {
            "messages": [AIMessage(content="Writing Lead: Sending to Technical Writer.")],
            "next_agent": "tech_writer",
            "current_task": task,
        }
    if not state.get("summary_done", False):
        return {
            "messages": [AIMessage(content="Writing Lead: Sending to Summary Writer.")],
            "next_agent": "summary_writer",
            "current_task": task,
        }

    # Both present -> return to CEO for finalization
    return {
        "messages": [AIMessage(content="Writing Lead: Draft & summary ready. Returning to CEO.")],
        "next_agent": "ceo",
    }

In [14]:
def tech_writer(state: OrgState) -> dict:
    task = state.get("current_task", "")
    prompt = (
        "You are a Technical Writer. Convert the research synthesis into a structured,\n"
        "well-cited technical write-up with headings and bullet points where useful.\n"
        "Audience: domain practitioners. 300-400 words.\n\n"
        f"Task: {task}\n\n"
        f"Research Synthesis:\n{state.get('research_synthesis','')}\n"
    )
    out = llm.invoke([HumanMessage(content=prompt)]).content
    return {
        "messages": [AIMessage(content="Technical Writer: Produced technical draft.")],
        "tech_draft": out,
        "tech_draft_done": True,
        "next_agent": "writing_lead",
    }

In [15]:
def summary_writer(state: OrgState) -> dict:
    task = state.get("current_task", "")
    prompt = (
        "You are a Summary Writer. Produce an executive summary (120-180 words)\n"
        "for senior leadership. Focus on ROI, risk, timeline. Include 3 bullets\n"
        "with clear recommendations.\n\n"
        f"Task: {task}\n\n"
        f"Research Synthesis:\n{state.get('research_synthesis','')}\n"
    )
    out = llm.invoke([HumanMessage(content=prompt)]).content
    return {
        "messages": [AIMessage(content="Summary Writer: Produced executive summary." )],
        "summary_text": out,
        "summary_done": True,
        "next_agent": "writing_lead",
    }

In [16]:
from typing import get_args

def router(state: OrgState) -> Literal[
    "ceo",
    "research_lead",
    "data_researcher",
    "market_researcher",
    "writing_lead",
    "tech_writer",
    "summary_writer",
    END,
]:
    nxt = state.get("next_agent") or "ceo"
    if nxt == "end" or state.get("task_complete", False):
        return END
    allowed = {
        "ceo",
        "research_lead",
        "data_researcher",
        "market_researcher",
        "writing_lead",
        "tech_writer",
        "summary_writer",
    }
    return nxt if nxt in allowed else "ceo"

In [17]:
workflow = StateGraph(OrgState)

workflow.add_node("ceo", ceo)
workflow.add_node("research_lead", research_lead)
workflow.add_node("data_researcher", data_researcher)
workflow.add_node("market_researcher", market_researcher)
workflow.add_node("writing_lead", writing_lead)
workflow.add_node("tech_writer", tech_writer)
workflow.add_node("summary_writer", summary_writer)

workflow.set_entry_point("ceo")

for node in [
    "ceo",
    "research_lead",
    "data_researcher",
    "market_researcher",
    "writing_lead",
    "tech_writer",
    "summary_writer",
]:
    workflow.add_conditional_edges(
        node,
        router,
        {
            "ceo": "ceo",
            "research_lead": "research_lead",
            "data_researcher": "data_researcher",
            "market_researcher": "market_researcher",
            "writing_lead": "writing_lead",
            "tech_writer": "tech_writer",
            "summary_writer": "summary_writer",
            END: END,
        },
    )

graph = workflow.compile()

In [18]:
if __name__ == "__main__":
    init_state: OrgState = {
        "messages": [
            HumanMessage(content=(
                "Evaluate the viability of launching an AI-enabled telehealth triage tool "
                "for mid-sized US clinics."
            ))
        ],
        "next_agent": None,
        "current_task": "",
        "data_notes": "",
        "market_notes": "",
        "data_done": False,
        "market_done": False,
        "research_synthesis": "",
        "research_synthesized": False,
        "tech_draft": "",
        "tech_draft_done": False,
        "summary_text": "",
        "summary_done": False,
        "final_report": "",
        "task_complete": False,
    }

    result = graph.invoke(init_state)

    print("\n--- PIPELINE COMPLETE ---\n")
    print((result.get("final_report") or "<no final report>")[:2000])

/opt/homebrew/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=e5f8fd85-0b47-447d-83f8-f50561f77f20; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=67281e1b-24a5-42e0-a12f-caceed8b8da3; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=f7146b6f-dde4-4798-a15d-9251f4490de1; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=eb502c30-7b77-4d01-830d-9c0ee79cdd1f; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=55d5f6e3-e28b-466d-81d0-c6c4ad4ffbe8; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=aa150

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=37960851-a501-4637-8d3d-7df7b26fa965; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=728adc78-9001-4469-8e6b-1763291450da; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=728adc78-9001-4469-8e6b-1763291450da; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=e98adf24-5551-4a2e-9058-08371c6a8127; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=1c870514-8ae9-42bf-bdd7-60210364df70; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=9d012b48-9d8a-471e-8b87-093935f862d5; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=9d012b48-9d8a-471e-8b87-093935f862d5; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id=1c870514-8ae9-42bf-bdd7-60210364df70; trace=1bb6d9ea-b91f-49fa-90d7-52b815f97aa8,id